In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

### 数据清理

In [2]:
df = pd.read_csv("../data/sample_data.csv")

In [3]:
# 汇总省级数据
province_summary = df.groupby(['province'], as_index=False).agg({
    'sales_rmb': 'sum',
    'revenue_rmb': 'sum'
})

# 汇总城市级数据，保留省级信息
city_summary = df.groupby(['province', 'city'], as_index=False).agg({
    'sales_rmb': 'sum',
    'revenue_rmb': 'sum'
})

# 汇总区域级数据，保留省级和城市级信息
district_summary = df.groupby(['province', 'city', 'district'], as_index=False).agg({
    'sales_rmb': 'sum',
    'revenue_rmb': 'sum'
})

# 创建一个Excel writer对象，保存三个sheet
with pd.ExcelWriter("../docs/sample/A_summary_result.xlsx") as writer:
    province_summary.to_excel(writer, sheet_name="Province Summary", index=False)
    city_summary.to_excel(writer, sheet_name="City Summary", index=False)
    district_summary.to_excel(writer, sheet_name="District Summary", index=False)

print("Data cleaned and saved to 'summary_result.xlsx'")

Data cleaned and saved to 'summary_result.xlsx'


### 数据矫正
shapefile投影矫正到84

In [4]:
# 加载空间数据（省级、市级、区级轮廓的GeoJSON文件）
province_gdf = gpd.read_file("../data/raw/province_geo.geojson")  # 省级GeoJSON
city_gdf = gpd.read_file("../data/raw/city_geo.geojson")          # 市级GeoJSON
district_gdf = gpd.read_file("../data/raw/district_geo.geojson")  # 区级GeoJSON

# 将所有的GeoDataFrame投影到WGS 84 (EPSG:4326)
province_gdf = province_gdf.to_crs(epsg=4326)
city_gdf = city_gdf.to_crs(epsg=4326)
district_gdf = district_gdf.to_crs(epsg=4326)

In [5]:
# # 读取 geojson 文件
# geojson_file = "./shp/district_geo.geojson"
# gdf = gpd.read_file(geojson_file)

# # 打印列名
# print("列名:", gdf.columns.tolist())

### 空间连接

In [6]:
# 将 lat 和 lon 转换为 GeoDataFrame 中的几何列
geometry = [Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# 4. 确保数据的 CRS 设置为 WGS 84 (EPSG:4326)，如果数据没有 CRS 信息
gdf = gdf.set_crs(epsg=4326, allow_override=True)

# 5. 空间连接：通过空间连接将省级、市级、区级的轮廓与数据连接
province_join = gpd.sjoin(gdf, province_gdf, how="left", predicate="within")
city_join = gpd.sjoin(gdf, city_gdf, how="left", predicate="within")
district_join = gpd.sjoin(gdf, district_gdf, how="left", predicate="within")

# 6. 检查是否有错误：比较空间连接结果和原数据中的省、市、区信息
province_join['error_province'] = (province_join['province'] != province_join['name']).astype(int)
city_join['error_city'] = (city_join['city'] != city_join['name']).astype(int)
district_join['error_district'] = (district_join['district'] != district_join['name']).astype(int)

# 7. 合并结果，生成新的列（省、市、区的空间连接结果和错误标记）
result = df.copy()
result['province_geo'] = province_join['name']
result['city_geo'] = city_join['name']
result['district_geo'] = district_join['name']
result['error_flag'] = (province_join['error_province'] | city_join['error_city'] | district_join['error_district']).astype(int)

# 8. 保存为新的Excel文件
result.to_excel("../docs/sample/spatial_join_results.xlsx", index=False)

print("空间连接完成，结果已保存到'spatial_join_results.xlsx'")

空间连接完成，结果已保存到'spatial_join_results.xlsx'


### 建立geojson （轮廓+交易额）

In [7]:
# 导入矫正之后的csv
# 可能在此之前还要加一段修正代码，但也可能是人为修正，具体看后面的数据来补代码
# df = pd.read_csv("")

In [8]:
from shapely.geometry import Polygon

# 汇总省级数据
province_summary = df.groupby(['province'], as_index=False).agg({
    'sales_rmb': 'sum',
    'revenue_rmb': 'sum'
})

# 汇总城市级数据，保留省级信息
city_summary = df.groupby(['province', 'city'], as_index=False).agg({
    'sales_rmb': 'sum',
    'revenue_rmb': 'sum'
})

# 汇总区域级数据，保留省级和城市级信息
district_summary = df.groupby(['province', 'city', 'district'], as_index=False).agg({
    'sales_rmb': 'sum',
    'revenue_rmb': 'sum'
})

# 结合汇总数据与对应的GeoJSON文件
# 假设省级数据的GeoJSON文件包含一个名为 'name' 的字段，城市和区级文件也是如此
# 将汇总数据与省、市、区的几何信息合并

# 合并省级数据
province_gdf = province_gdf.merge(province_summary, left_on="name", right_on="province", how="left")

# 合并城市级数据
city_gdf = city_gdf.merge(city_summary, left_on="name", right_on="city", how="left")

# 合并区级数据
district_gdf = district_gdf.merge(district_summary, left_on="name", right_on="district", how="left")

# 保存为GeoJSON文件
province_gdf.to_file("../data/shp/province_summary.geojson", driver="GeoJSON")
city_gdf.to_file("../data/shp/city_summary.geojson", driver="GeoJSON")
district_gdf.to_file("../data/shp/district_summary.geojson", driver="GeoJSON")

print("GeoJSON files for province, city, and district summaries saved.")

GeoJSON files for province, city, and district summaries saved.


---

In [9]:
# import geopandas as gpd
# import pandas as pd

# def safe_deduplicate_with_null_protection(input_file, output_file):
#     print(f"正在分析文件: {input_file}")
#     gdf = gpd.read_file(input_file)
    
#     # 1. 定义四维度字段
#     cols = ['province', 'city', 'district', 'name']

#     # 2. 筛选出：四项全都有值 的数据
#     # notnull().all(axis=1) 确保这四个字段一个 null 都没有
#     complete_data_mask = gdf[cols].notnull().all(axis=1)
#     gdf_complete = gdf[complete_data_mask].copy()
    
#     # 3. 在完整数据中找重复
#     # 使用 keep=False 标记所有参与重复的行
#     dup_mask = gdf_complete.duplicated(subset=cols, keep=False)
    
#     if dup_mask.any():
#         to_dissolve = gdf_complete[dup_mask].copy()
#         # 剩下的数据：包括包含 null 的行，以及不重复的完整行
#         to_keep_as_is = gdf[~complete_data_mask | (complete_data_mask & ~dup_mask)].copy()

#         print("\n" + "!"*60)
#         print(f"发现以下【四项全满且完全一致】的重复项，准备进行合并：")
#         print(to_dissolve[cols + ['gb', 'revenue_rmb']].to_string(index=False))
#         print("!"*60 + "\n")

#         # 4. 仅对重复项进行聚合
#         # 按四维度聚合，累加业务数值
#         dissolved_part = to_dissolve.dissolve(by=cols, aggfunc={
#             'revenue_rmb': 'sum',
#             'sales_rmb': 'sum',
#             'gb': 'first' # gb 码保留遇到的第一个
#         }).reset_index()

#         # 5. 合并回原始数据（保留了所有 null 行和唯一行）
#         final_gdf = pd.concat([to_keep_as_is, dissolved_part], ignore_index=True)
        
#         print(f"合并完成：原始重复行已合成为 {len(dissolved_part)} 条记录。")
#     else:
#         print("\n未发现【四项全满且完全一致】的重复记录，保持原样。")
#         final_gdf = gdf

#     # 保存
#     final_gdf.to_file(output_file, driver='GeoJSON')
#     print(f"\n最终统计：")
#     print(f"原始记录总数: {len(gdf)}")
#     print(f"去重后总数: {len(final_gdf)}")
#     print(f"结果已存至: {output_file}")

# # 执行
# safe_deduplicate_with_null_protection('district_summary.geojson', 'district_summary_fixed.geojson')